In [ ]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
#install packages
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [13]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Access the connection string using the correct key
try:
    connection_string = config["connectionString"]
    print("Connection String:", connection_string)
except KeyError:
    print("The key 'connectionString' does not exist in the configuration.")
    connection_string = None

# Check if connection_string is not None before proceeding
if connection_string:
    # Set up connection string for Azure storage and container name
    container_azure = 'metro2023data'

    # Initialize the BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client(container_azure)

    # Initialize an empty list to store DataFrames
    metro_dataframes = []

    # List all blobs in the specified container
    blob_list = container_client.list_blobs()
    for blob in blob_list:
        print(blob.name)
        blob_client = container_client.get_blob_client(blob=blob.name)
        blob_data = blob_client.download_blob()
        blob_content = blob_data.readall().decode('utf-8')
        df = pd.read_csv(StringIO(blob_content))
        # Display the head of the DataFrame
        print(df.head())
        # Append the DataFrame to the list
        metro_dataframes.append(df)

    # Now you have a list of DataFrames, each corresponding to a blob
    # Example: Access the first DataFrame
    if metro_dataframes:
        first_metro_df = metro_dataframes[0]
        print("First DataFrame from the list:")
        print(first_metro_df.head())
else:
    print("Failed to retrieve connection string from configuration.")


Connection String: DefaultEndpointsProtocol=https;AccountName=metro2023;AccountKey=idxS7hOlQ/5Ks3LVuMVW5wnypDFc28oB7qu8CJ+EKG5V9R/126GSkcH98dQeXdbdGLIFS25cYH/G+AStmajdcw==;EndpointSuffix=core.windows.net
metro2023data.csv
     trip_id  duration     start_time       end_time  start_station  \
0  276590229         1  7/1/2023 0:01  7/1/2023 0:02           4215   
1  276590710        16  7/1/2023 0:02  7/1/2023 0:18           4494   
2  276613156       148  7/1/2023 0:02  7/1/2023 2:30           4215   
3  276590579         3  7/1/2023 0:03  7/1/2023 0:06           4215   
4  276613157       146  7/1/2023 0:04  7/1/2023 2:30           4215   

   start_lat   start_lon  end_station    end_lat     end_lon bike_id  \
0  34.014309 -118.491341         4215  34.014309 -118.491341   30096   
1  34.075981 -118.256012         3031  34.044701 -118.252441   24279   
2  34.014309 -118.491341         4215  34.014309 -118.491341   30240   
3  34.014309 -118.491341         4215  34.014309 -118.491341   

<ipython-input-13-e418c13dc7b1>:37: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


In [64]:
# Extracting dim_time dimension
dim_time = first_metro_df[['start_time', 'end_time']]

# Renaming columns
dim_time.columns = ['start_time', 'end_time']

# Adding time_id column
dim_time['time_id'] = dim_time.index + 1

# Reorder columns
dim_time = dim_time[['time_id', 'start_time', 'end_time']]

dim_time.head()

# Save to Desktop
from google.colab import files
files.download('dim_time.csv')

<ipython-input-64-7138ce51948a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_time['time_id'] = dim_time.index + 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
# Extracting dim_passholder dimension
dim_passholder = df[['passholder_type']].copy().drop_duplicates().reset_index(drop=True)

# Adding passholder_id column
dim_passholder['passholder_id'] = dim_passholder.index + 1

# Reorder columns
dim_passholder = dim_passholder[['passholder_id', 'passholder_type']]

dim_passholder.head()

# Save to Desktop
dim_passholder.to_csv('dim_passholder.csv', index=False)
from google.colab import files
files.download('dim_passholder.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
# Extracting dim_station dimension
start_stations = df[['start_station', 'start_lat', 'start_lon']].rename(columns={'start_station': 'station_id', 'start_lat': 'latitude', 'start_lon': 'longitude'})
end_stations = df[['end_station', 'end_lat', 'end_lon']].rename(columns={'end_station': 'station_id', 'end_lat': 'latitude', 'end_lon': 'longitude'})

# Concatenate and remove duplicates
dim_station = pd.concat([start_stations, end_stations]).drop_duplicates().reset_index(drop=True)

# Adding stationname column as a placeholder
dim_station['stationname'] = "Station " + dim_station['station_id'].astype(str)
dim_station['station_id'] = dim_station['stationname']

# Reorder columns
dim_station = dim_station[['station_id', 'latitude', 'longitude']]

dim_station.head()

# Save to Desktop
dim_passholder.to_csv('dim_station.csv', index=False)
from google.colab import files
files.download('dim_station.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
# Extract dates
start_dates = pd.to_datetime(df['start_time']).dt.date
end_dates = pd.to_datetime(df['end_time']).dt.date

# Concatenate and remove duplicates, convert to datetime
all_dates = pd.to_datetime(pd.concat([start_dates, end_dates]).drop_duplicates().reset_index(drop=True))

# Create dim_date DataFrame
dim_date = pd.DataFrame({
    'date_id': all_dates.index + 1,
    'date': all_dates,
    'year': all_dates.dt.year,
    'quarter': all_dates.dt.quarter,
    'month': all_dates.dt.month,
    'day': all_dates.dt.day,
    'weekday': all_dates.dt.weekday + 1  # Monday=1, Sunday=7
})

dim_date.head()

# Save to Desktop
dim_passholder.to_csv('dim_date.csv', index=False)
from google.colab import files
files.download('dim_date.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
# Extracting fact_bike_trips
fact_bike_trips = df[['trip_id', 'duration', 'start_station', 'bike_id', 'passholder_type', 'trip_route_category', 'bike_type']].copy()

# Map passholder_type to passholder_id
passholder_mapping = dim_passholder.set_index('passholder_type')['passholder_id'].to_dict()
fact_bike_trips['passholder_id'] = fact_bike_trips['passholder_type'].map(passholder_mapping)

# Convert start_time to datetime and extract time_id
fact_bike_trips['start_time'] = pd.to_datetime(df['start_time'])
fact_bike_trips['time_id'] = fact_bike_trips.index + 1

# Map start_station to station_id
fact_bike_trips = fact_bike_trips.rename(columns={'start_station': 'station_id'})

# Add fact_id and data_id columns
fact_bike_trips['fact_id'] = fact_bike_trips.index + 1
fact_bike_trips['data_id'] = fact_bike_trips.index + 1  # Assuming data_id is just a unique identifier

# Rename columns
fact_bike_trips = fact_bike_trips.rename(columns={
    'duration': 'trip_duration',
    'trip_route_category': 'trip_route_category',
    'bike_type': 'bike_type'
})

# Reorder columns to match fact table structure
fact_bike_trips = fact_bike_trips[['fact_id', 'trip_id', 'time_id', 'station_id', 'bike_id', 'passholder_id', 'trip_duration', 'trip_route_category', 'bike_type', 'data_id']]


fact_bike_trips.head()

# Save to Desktop
dim_passholder.to_csv('fact_bike_trips.csv', index=False)
from google.colab import files
files.download('fact_bike_trips.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>